In [1]:
from bs4 import BeautifulSoup
import requests
import time 
import random 
from tqdm import tqdm 
from collections import defaultdict 
import json
import pandas as pd

In [2]:
#pd.set_option('display.max_rows', None) 
pd.options.display.float_format = '{:.5f}'.format

In [224]:
# 쿠팡 밀키트 데이터 불러오기
coupang = pd.read_csv('coucoupangpang.csv', index_col = 0)

In [4]:
# 요기요 카테고리 페이지에서 가게정보 json 파일 크롤링
def yogiyo_page_crawling(geo_url, api_key, parameters, headers):
    res = requests.get(geo_url, params=parameters, headers=headers)
    page = json.loads(res.content)
    return page

# json파일에서 필요한 정보만 데이터프레임화
def yogiyo_dataframe(main_page):
    key_list = ["id", "name", "address", "review_count", "categories"]
    df_list = []
    for idx in range(len(main_page["restaurants"])):
        info_list= []
        for key in key_list:
            info_list.append(main_page["restaurants"][idx][key])
        df_list.append(info_list)
    df = pd.DataFrame(df_list , columns = key_list)
    return df

# 각 가게의 세부정보 페이지 json 파일 크롤링
def yogiyo_product_crawling(df, product_url, api_key, parameters, headers):
    df.id = df.id.astype(str)
    pageid = list(df.id.values)
    all_info = []
    # n개의 가게만 뽑아오기
    for num in range(10):
        detailed_url = product_url + pageid[num] + '/menu/'        
        res = requests.get(detailed_url,params=parameters, headers=headers)
        info = json.loads(res.content)
        all_info.append(info)
    return all_info


# 요기요 점포 (메뉴 , 가격) 리스트화
def yogiyo_menu_price(df, page):
    cnt = 0
    menu_df = pd.DataFrame(columns=['idx', 'menu_idx', 'menu_name', 'price'], index = [0])
    for h in range(len(page)):
        for i in range(len(page[h])):
            for j in range(len(page[h][i].get('items'))):
                idx = df['name'][h]
                menu_idx = page[h][i].get('items')[j].get('section')
                '''
                if page[h][i].get('items')[j].get('section') == 'top_items':
                    menu_idx = page[h][i].get('items')[j].get('section')
                else :
                    continue
                '''
                menu_name = page[h][i].get('items')[j].get('name')
                price = page[h][i].get('items')[j].get('price')
                menu_df.loc[cnt] = [idx, menu_idx, menu_name, price]
                cnt += 1
    return menu_df


In [ ]:
# 요기요 - 쿠팡 밀키트 가격 비교에 사용할 사전
 
yogiyo_dict = {'곱창전골' : ['곱창전골', '곱창 전골'],
               '나베' : ['나베'],
               '샤브샤브' : ['샤브샤브'],
               '부대전골' : ['부대전골'],
               '불고기전골' : ['불고기전골'],
               '낙곱새' : ['낙곱새'],
               '만두전골' : ['만두전골'],
               '스키야키' : ['스키야키'],
               
               '순두부찌개' : ['순두부찌개', '순두부 찌개'],
               '된장찌개' : ['된장찌개'],
               '청국장찌개' : ['청국장'],
               '고추장찌개' : ['고추장찌개'],
               '김치찌개' : ['김치찌개'],
               '부대찌개' : ['부대찌개'],
               '돼지고기짜글이' : ['삼겹짜글이', '돼지고기 김치짜글이', '돼지짜글이', '돼지김치짜글이'],
               
               '투움바파스타' : ['투움바 파스타'],
               '크림파스타' : ['크림파스타', '크림 파스타'],
               '로제파스타' : ['로제파스타', '로제 파스타'],
               '봉골레파스타' : ['봉골레'],
               '해장파스타' : ['해장파스타', '해장 파스타'],
               '토마토파스타' : ['토마토파스타', '토마토 파스타'],
               '냉파스타' : ['냉파스타'],
               '바지락파스타' : ['바지락 파스타'],
               '새우파스타' : ['새우 파스타', '쉬림프파스타', '쉬림프 파스타'],
               '오일파스타' : ['오일파스타', '오일 파스타'],
               
               '메기탕' : ['메기매운탕'],
               '대구탕' : ['대구탕', '대구지리', '대구매운', '대구뽈탕', '대구머리'],
               '아구탕' : ['아구탕', '아구 매운탕', '아구 지리탕', '아구지리'],
               '동태탕' : ['동태탕', '동태내장탕'],
               '우럭탕' : ['우럭탕', '우럭매운탕', '우럭 매운탕', '우럭 통매운탕', '우럭맑은지리'],

               '조개탕' : ['조개탕'],
               '낙지탕' : ['연포', '낙지탕'],
               '누룽지탕' : ['해물누룽지탕', '해물 누룽지탕'],
               '꽃게탕' : ['꽃게탕'],
               '홍합탕' : ['홍합탕', '홍합짬뽕탕'],
               '새우탕' : ['새우탕'],
               '해물탕' : ['해물탕'],
               
               '찹스테이크' : ['찹스테이크', '찹 스테이크'],
               '스테이크' : ['스테이크'],
               
               '조기구이' : ['조기구이', '제주도 참조기'],
               '갈치구이' : ['갈치구이'],
               '고등어구이' : ['고등어구이', '고등어 구이'],
               
               '감바스' : ['감바스'],
               
               '미역국' : ['미역국'],
               '순대국밥' : ['순대국밥'],
               '육개장' : ['육개장'],
               '떡국' : ['떡국'],
               '소고기무국' : ['소고기무국', '소고기 무국'],
               
               '닭발' : ['닭발'],
               '닭갈비' : ['닭갈비'],
               '백숙' : ['백숙'],
               '닭볶음탕' : ['닭볶음'],
               
               '우동' :['야끼우동','청춘우동','짬뽕우동','포장마차우동','사누끼우동','사누끼 우동','구르미우동','온우동','볶음우동','어묵우동','즉석우동','얼큰우동'],
               '탄탄면' : ['탄탄면'],
               '짬뽕' : ['짬뽕'],
               
               '콩불' : ['콩불'],
               '퀘사디아' : ['퀘사디아'],
               '갈치조림' : ['갈치조림','갈치 조림'],
               '순대볶음' : ['순대볶음','순대 볶음'],
               
               '고추잡채' : ['고추잡채','고추 잡채'],
               '숙주볶음' : ['숙주볶음','숙주 볶음'],
               
               '꼬막무침' : ['꼬막 무침','꼬막무침'],
               
               '알탕' : ['알탕','알곤이탕'],
               
               '칼국수' : ['칼국수'],
               '비빔국수' : ['비빔국수'],
               '쌀국수' : ['쌀국수'],
               '초계국수' : ['초계국수'],
               
               '떡볶이' : ['떡볶이'],
               
               '아귀찜' : ['아귀찜','아구찜'],
               '대구뽈찜' : ['대구뽈찜','대구왕뽈찜'],
               '술찜' : ['술찜'],
               '묵은지찜' : ['묵은지 김치찜（돼지）','돼지김치찜','삼겹 김치찜','흑돈 김치찜'],
               
               '마라탕' : ['마라탕'],
               
               '도가니탕' : ['도가니탕'],
               '어묵탕' : ['어묵탕','오뎅탕'],
               '김치찜' : ['묵은지 김치찜（돼지）','돼지김치찜','삼겹 김치찜','흑돈 김치찜'],
               '양장피' : ['양장피']
               }

In [26]:
# 실행 코드
'''
category_list = ['고기구이', '피자양식', '야식', '치킨', '테이크아웃', 'SNS맛집', '신규맛집', '분식', '찜탕', '한식', '샌드위치', '샐러드', '카페디저트', 
                 '아시안', '버거', '족발보쌈', '회초밥', '도시락죽', '중식', '일식돈까스', '예약픽업']
'''        
category_list = ['고기구이', '피자양식', '야식', '분식', '찜탕', '한식', '샌드위치', 
                 '샐러드', '카페디저트', '아시안', '족발보쌈', '도시락죽', '중식', '일식돈까스']
         
spot = pd.read_excel('시군구위도경도.xlsx')[0:25]
lat_list = list(spot.y.values)
lng_list = list(spot.x.values)

geo_url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/'
product_url = 'https://www.yogiyo.co.kr/api/v1/restaurants/'
api_key = 'fe5183cc3dea12bd0ce299cf110a75a2'

yogiyo_menu = pd.DataFrame(columns = ['idx','menu_idx','menu_name','price'])#, index = [0])
#for category in category_list:
for i in range(len(spot)):
    parameters = {"category" : '일식돈까스',
                    #"items" : 20 ,
                    "lat" : lat_list[i],
                    "lng" : lng_list[i],
                    "order" : 'review_count',
                    "page" : 0
                    }
    headers = {'X-ApiSecret':api_key,'X-ApiKey':"iphoneap"}

    yogiyo_page = yogiyo_page_crawling(geo_url, api_key, parameters, headers)
    yogiyo_df = yogiyo_dataframe(yogiyo_page)
    yogiyo_product_page = yogiyo_product_crawling(yogiyo_df, product_url, api_key, parameters, headers)
    menu = yogiyo_menu_price(yogiyo_df, yogiyo_product_page)
    yogiyo_menu = pd.concat([yogiyo_menu, menu])


# 메모리 이슈 & 사전화에 용이하게끔 각 카테고리별로 csv파일에 저장
yogiyo_menu.price = yogiyo_menu.price.astype(int)
yogiyo_menu = yogiyo_menu.reset_index().drop('index', axis = 1)
yogiyo_menu.to_csv(parameters["category"]+'.csv')

In [277]:
고기구이 = pd.read_csv('고기구이.csv', index_col= 0)
피자양식 = pd.read_csv('피자양식.csv', index_col= 0)
야식 = pd.read_csv('야식.csv', index_col= 0)
분식 = pd.read_csv('분식.csv', index_col= 0)
찜탕 = pd.read_csv('찜탕.csv', index_col= 0)
한식 = pd.read_csv('한식.csv', index_col= 0)
샌드위치 = pd.read_csv('샌드위치.csv', index_col= 0)
샐러드 = pd.read_csv('샐러드.csv', index_col= 0)
카페디저트 = pd.read_csv('카페디저트.csv', index_col= 0)
아시안 = pd.read_csv('아시안.csv', index_col= 0)
족발보쌈 = pd.read_csv('족발보쌈.csv', index_col= 0)
도시락죽 = pd.read_csv('도시락죽.csv', index_col= 0)
중식 = pd.read_csv('중식.csv', index_col= 0)
일식돈까스 = pd.read_csv('일식돈까스.csv', index_col= 0)

# 모든 카테고리 데이터프레임 병합
concat_df = pd.concat([고기구이, 피자양식, 야식, 분식, 찜탕, 한식, 샌드위치, 샐러드, 카페디저트, 아시안, 족발보쌈, 도시락죽, 중식, 일식돈까스], ignore_index= True)
concat_df.to_csv('yogiyo_dataset.csv')

# 병합한 데이터프레임 간단한 전처리
data = pd.read_csv('yogiyo_dataset.csv', index_col= 0)
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis = 1, inplace= True)
data

,idx,menu_idx,menu_name,price
0,전곱-한성대직영점,top_items,야채곱창,26000
1,전곱-한성대직영점,top_items,순대곱창,29000
2,전곱-한성대직영점,top_items,야채곱창 1인분 실속형,20000
3,전곱-한성대직영점,top_items,돼지막창소금구이,31000
4,전곱-한성대직영점,top_items,곱창막창볶음,31000
...,...,...,...,...
165338,우마이돈가스,SIDE 메뉴,반 냉모밀,4000
165339,우마이돈가스,SIDE 메뉴,미니 돈가스,4000
165340,우마이돈가스,SIDE 메뉴,왕새우 튀김,5000
165341,우마이돈가스,SIDE 메뉴,공기밥,1000


In [278]:
# 데이터프레임과 사전의 text를 비교해서 필요한 내용만 추출하는 함수
def dict_to_dataframe(dataframe, dict, column):
    menu_list = []
    for idx, product in enumerate(dataframe[column]):
        sublist = []
        for key in list(dict.keys()):
            for i in dict[key]:
                if i in product:
                    sublist.append(key)
        # 중복 제거 
        sublist = list(set(sublist))
        sublist = ', '.join(sublist)
        menu_list.append(sublist)
    return menu_list

In [279]:
# data의 menu_name 컬럼에서, 사전의 value값에 해당하는 text가 존재한다면 사전의 key를 원소롤 받음
# 위 함수에서 뽑아낸 리스트를 시리즈로 변환 -> data dataframe에 추가
yogiyo_list = dict_to_dataframe(data, yogiyo_dict, 'menu_name')
yogiyo_series = pd.Series(yogiyo_list)
data['menu_edited'] = yogiyo_series

In [280]:
data

,idx,menu_idx,menu_name,price,menu_edited
0,전곱-한성대직영점,top_items,야채곱창,26000,
1,전곱-한성대직영점,top_items,순대곱창,29000,
2,전곱-한성대직영점,top_items,야채곱창 1인분 실속형,20000,
3,전곱-한성대직영점,top_items,돼지막창소금구이,31000,
4,전곱-한성대직영점,top_items,곱창막창볶음,31000,
...,...,...,...,...,...
165338,우마이돈가스,SIDE 메뉴,반 냉모밀,4000,
165339,우마이돈가스,SIDE 메뉴,미니 돈가스,4000,
165340,우마이돈가스,SIDE 메뉴,왕새우 튀김,5000,
165341,우마이돈가스,SIDE 메뉴,공기밥,1000,


In [281]:
# 유사한 음식이 묶인 menu_edited 컬럼을 기준으로 그룹화후, 평균 price 값 추출 및 반올림
grouped_data = data.groupby(data['menu_edited'])
price_mean = grouped_data.price.mean()
roundd = price_mean.values.round(-2)
price_mean = price_mean.round(-2)

In [282]:
# menu_edited의 결측값(yogiyo_dict의 value와 일치하는 값이 없는 메뉴들)제거
price_mean = price_mean.drop([''])
price_mean

menu_edited
갈치구이             11800.00000
갈치조림             18800.00000
감바스              19000.00000
고등어구이            12700.00000
고추잡채             34100.00000
고추잡채, 양장피        90500.00000
고추장찌개            10300.00000
곱창전골             30800.00000
김치찌개             15400.00000
꼬막무침             15600.00000
꽃게탕              29900.00000
나베               13800.00000
낙곱새              21600.00000
낙지탕              32100.00000
낙지탕, 샤브샤브        50000.00000
냉파스타              9400.00000
누룽지탕             25200.00000
누룽지탕, 양장피        50000.00000
닭갈비              16900.00000
닭발               21700.00000
닭발, 떡볶이          24000.00000
닭볶음탕             24400.00000
대구뽈찜             32100.00000
대구탕              18400.00000
도가니탕             17700.00000
동태탕              19600.00000
동태탕, 아귀찜         28600.00000
돼지고기짜글이          16000.00000
된장찌개             11700.00000
떡국                8300.00000
떡국, 떡볶이          14400.00000
떡국, 부대찌개         16500.00000
떡볶이              12200.00000
떡볶이, 닭갈비         25000.00000
떡볶

In [283]:
# 만들어 놓은 함수를 활용하여, coupang의 '상품' 컬럼에서, 사전의 value값에 해당하는 text가 존재한다면 사전의 key를 원소롤 받음
coupang_list = dict_to_dataframe(coupang, yogiyo_dict, '상품')
coupang_series = pd.Series(coupang_list)
coupang['menu_edited'] = coupang_series

In [284]:
# price_mean에서 추출한 메뉴별 평균 가격을 coupang 데이터에 apply
# 이 때 price_mean.index에 없는 x의 경우에는, split으로 각 메뉴를 분리후 각각의 가격을 합계해주어 리턴해주는 방식
def func(x) :
    if x == '' :
        return 0
    if x in list(price_mean.index) :
        return price_mean[x]
    else :
        total_price = 0
        a = x.split(', ')
        for i in a:
            total_price += price_mean[i]
        return total_price
coupang["yogiyo_price"] = coupang["menu_edited"].apply(lambda x : func(x))

In [286]:
# 필요없는 컬럼을 제거, 시각화에 유리하게끔 yogiyo_price를 float -> int 로 형 변환
coupang.drop('menu_edited', axis = 1, inplace= True)
coupang.yogiyo_price = coupang.yogiyo_price.astype(int)

In [287]:
coupang

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,알레르기,yogiyo_price
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.50000,11454,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골,"소, 콩, 밀가루",30800
1,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.50000,15942,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골,"소, 콩, 밀가루",13800
2,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.50000,49899,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎",전골,"소, 밀가루",21500
3,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.00000,37085,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골,"돼지, 우유, 소, 밀가루",23200
4,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,20%,11900,"(100g당 1,469원)",4.50000,3073,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스",한식,"소, 콩",13900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,7,6631712092,15121133848,82342870262,쿡킷 스페인식 감바스 2인분 (냉동),"쿡킷 스페인식 감바스 2인분 (냉동), 315g, 1개",12500,0,12500,"(100g당 3,968원)",4.50000,276,0,"구성품1(올리브유, 파슬리 후레이크, 크러쉬드 레드페퍼 등 함유) , 구성품2(마늘...",올리브유,감바스,"갑각류, 밀가루",19000
186,7,6188303158,12197744800,79468743704,프레시지 박막례 치즈 떡볶이 (냉동),"프레시지 박막례 치즈 떡볶이 (냉동), 485g, 1개",6800,4%,6500,"(100g당 1,340원)",4.50000,2089,0,"어묵, 소스, 대파, 떡으로 구성되어 있습니다.","어묵,소스,대파,떡",떡볶이,"우유, 생선류",12200
187,7,6635938377,15150332499,82371861026,쿡킷 동대문식 닭한마리 2인분 (냉동),"쿡킷 동대문식 닭한마리 2인분 (냉동), 1150g, 1개",17910,5%,16900,"(100g당 1,470원)",4.50000,279,0,"양념 닭고기, 채소(감자, 대파), 다진 마늘, 떡과 칼국수 면, 치킨스톡으로 구성...","채소,감자,대파,다진마늘,떡,칼국수면,치킨스톡",닭,"닭, 밀가루",0
188,7,6641444058,15183076854,82404328814,마이셰프 TREASURE HUNTER 우정분식 꿀꺽 야끼만두 2인분 (냉동),"마이셰프 TREASURE HUNTER 우정분식 꿀꺽 야끼만두 2인분 (냉동), 1팩...",5900,0,5900,"(100g당 2,458원)",4.00000,10,0,"잡채 튀김, 소스로 구성되어 있습니다.",소스,기타,밀가루,0
